# Proyecto SQL

En este proyecto analizaremos una base de datos de libros para:

- Entender la producción editorial después del año 2000

- Evaluar la calidad de los libros (reseñas y calificaciones)

- Identificar editoriales y autores destacados

- Analizar el comportamiento de los usuarios más activos

In [ ]:
# importar librerías y conexion a la base de datos
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
query_1 = """SELECT COUNT(*) AS num_books_after_2000
FROM books
WHERE publication_date > '2000-01-01';"""

pd.io.sql.read_sql(query_1, con = engine)

,num_books_after_2000
0,819



Se publicaron 819 libros despues del 1ro de enero del 2000 

In [22]:
query_2 = """
SELECT 
    b.title,
    COUNT(r.rating_id) AS num_ratings,
    ROUND(AVG(r.rating), 2) AS avg_rating
FROM books b
JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title
ORDER BY num_ratings DESC;
"""
df2 = pd.io.sql.read_sql(query_2, con = engine)
display(df2.head())
display(df2[['num_ratings', 'avg_rating']].agg(['min', 'max']))


,title,num_ratings,avg_rating
0,Twilight (Twilight #1),160,3.66
1,The Hobbit or There and Back Again,88,4.13
2,The Catcher in the Rye,86,3.83
3,Angels & Demons (Robert Langdon #1),84,3.68
4,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.41


,num_ratings,avg_rating
min,1,1.5
max,160,5.0


El libro mas rateado es Twilight, pero no la mejor calificada. Hay un empate en los libros mejores calificados (5.0) sin embargo el numero de reseñas es muy bajo (2). El libro que tiene una buena cantidad de reseñas y esta mejor calificado es Harry Potter y el prisionero de Azkaban

In [10]:
query_3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY num_books DESC
LIMIT 1;
"""
df3 = pd.io.sql.read_sql(query_3, con = engine)
display(df3)


,publisher,num_books
0,Penguin Books,42


La editorial Penguin Books es la editorial con mas titulos publicados con mas de 50 paginas

In [11]:
query_4 = """
SELECT 
    a.author,
    ROUND(AVG(r.rating), 2) AS avg_rating,
    COUNT(r.rating_id) AS total_ratings
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
GROUP BY a.author_id, a.author
HAVING COUNT(r.rating_id) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""
df4 = pd.io.sql.read_sql(query_4, con = engine)
display(df4)


,author,avg_rating,total_ratings
0,Diana Gabaldon,4.3,50


El/La autor/a que tiene la más alta calificación promedio de rating es Diana Gabaldon con un rating promedio de 4.3

In [13]:
query_5 = """
WITH users_with_50_ratings AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
),
text_reviews_per_user AS (
    SELECT username, COUNT(*) AS num_text_reviews
    FROM reviews
    WHERE text IS NOT NULL AND LENGTH(TRIM(text)) > 0
    GROUP BY username
)
SELECT 
    ROUND(AVG(t.num_text_reviews), 2) AS avg_text_reviews
FROM users_with_50_ratings u
LEFT JOIN text_reviews_per_user t ON u.username = t.username;
"""
df5 = pd.io.sql.read_sql(query_5, con = engine)
display(df5)

,avg_text_reviews
0,24.33


El promedio de reseñas con texto para usuarios con 50 o mas libros con reseña es de 24.33